<a href="https://colab.research.google.com/github/JonathanJuradoS/Planificacion_Heuristica/blob/main/Solucion_Caso_Integrador_CDAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ✅ Solución al Caso Práctico Integrador: Centro de Distribución Educativa (CDAA)

Este notebook contiene la resolución completa de las 5 operaciones clave usando algoritmos heurísticos:
1. Planificación Académica  
2. Logística Universitaria  
3. Producción de Kits Educativos  
4. Asignación de Personal  
5. Navegación Autónoma

Incluye análisis, heurísticas, algoritmos aplicados y conclusiones por sección.


In [1]:

import pandas as pd

# Cargar archivo Excel con múltiples hojas
archivo = "Dataset_Caso_Integrador_CDAA_3000.xlsx"
hojas = pd.read_excel(archivo, sheet_name=None)


## 1. Planificación Académica – Greedy Best-First Search

In [2]:

df = hojas["Planificacion_Academica"]

def greedy_planificacion(df):
    horario = pd.DataFrame(columns=df.columns)
    usados = set()
    for _, fila in df.iterrows():
        clave = (fila["Responsable"], fila["Fecha_Programada"])
        if clave not in usados:
            horario.loc[len(horario)] = fila
            usados.add(clave)
    return horario

horario_planificado = greedy_planificacion(df)
horario_planificado.head()


,ID_Operacion,Tipo_Operacion,Departamento,Sede,Responsable,Recurso_Asignado,Prioridad,Estado,Duracion_Estimada_h,Fecha_Programada
0,OP749487,Planificacion_Academica,Ingenieria,Arequipa,USR4521,Vehiculo,Baja,En Proceso,1,2025-08-06 00:00:00
1,OP834032,Planificacion_Academica,Logistica,Lima,USR4976,Operador,Baja,En Proceso,7,2025-08-07 00:00:00
2,OP848544,Planificacion_Academica,Ingenieria,Piura,USR6051,Aula,Media,Completado,5,2025-08-09 00:00:00
3,OP241888,Planificacion_Academica,Administracion,Arequipa,USR4809,Vehiculo,Baja,En Proceso,6,2025-08-05 00:00:00
4,OP844607,Planificacion_Academica,Educacion,Piura,USR2268,Operador,Baja,Completado,7,2025-08-22 00:00:00


## 2. Logística Universitaria – A*

In [4]:
import heapq

df = hojas["Logistica_Universitaria"]
sedes = df["Sede"].unique()

def heuristic_sede(a, b):
    return abs(hash(a) - hash(b)) % 10

def a_star_sedes(inicio, objetivo, sedes):
    frontera = [(0, inicio, [])]
    visitados = set()
    while frontera:
        costo, actual, camino = heapq.heappop(frontera)
        if actual == objetivo:
            return camino + [actual]
        if actual in visitados:
            continue
        visitados.add(actual)
        for s in sedes:
            if s != actual:
                nuevo_costo = costo + heuristic_sede(actual, s)
                heapq.heappush(frontera, (nuevo_costo, s, camino + [actual]))
    return []

ruta = a_star_sedes("Lima", "Arequipa", sedes)
ruta


['Lima', 'Arequipa']

## 3. Producción de Kits Educativos – Hill Climbing

In [6]:
# Importar random nuevamente ya que el entorno se ha reiniciado
import random

df = hojas["Produccion"]

def evaluar_orden(df, orden):
    return sum((i + 1) * df.loc[i, "Duracion_Estimada_h"] for i in orden)

def hill_climbing(df):
    orden = list(range(len(df)))
    random.shuffle(orden)
    mejor = orden.copy()
    mejor_score = evaluar_orden(df, mejor)
    for _ in range(500):
        i, j = random.sample(range(len(orden)), 2)
        nuevo = mejor.copy()
        nuevo[i], nuevo[j] = nuevo[j], nuevo[i]
        score = evaluar_orden(df, nuevo)
        if score < mejor_score:
            mejor, mejor_score = nuevo, score
    return df.loc[mejor]

orden_produccion = hill_climbing(df)
orden_produccion.head()


,ID_Operacion,Tipo_Operacion,Departamento,Sede,Responsable,Recurso_Asignado,Prioridad,Estado,Duracion_Estimada_h,Fecha_Programada
198,OP468239,Produccion,Sistemas,Cusco,USR3155,Aula,Media,Pendiente,3,2025-08-03
164,OP950366,Produccion,Logistica,Trujillo,USR8254,Docente,Baja,Completado,4,2025-08-19
522,OP778589,Produccion,Logistica,Arequipa,USR8991,Aula,Alta,En Proceso,7,2025-08-06
581,OP620787,Produccion,Administracion,Lima,USR6268,Docente,Baja,En Proceso,6,2025-08-31
493,OP630701,Produccion,Educacion,Arequipa,USR3757,Aula,Alta,Pendiente,5,2025-08-11


## 4. Asignación de Personal – Simulated Annealing

In [8]:
# Importar numpy que falta por el reinicio
import numpy as np

df = hojas["Asignacion_Personal"]

def penalizacion(asignacion):
    rep = asignacion["Responsable"].value_counts()
    return sum((c - 5) ** 2 for c in rep if c > 5)

def simulated_annealing(df):
    actual = df.sample(frac=1).reset_index(drop=True)
    mejor = actual.copy()
    mejor_score = penalizacion(mejor)
    temp = 100
    for _ in range(200):
        nuevo = actual.sample(frac=1).reset_index(drop=True)
        score = penalizacion(nuevo)
        if score < mejor_score or random.random() < np.exp((mejor_score - score) / temp):
            actual = nuevo
            mejor = nuevo
            mejor_score = score
        temp *= 0.95
    return mejor

asignacion_optimizada = simulated_annealing(df)
asignacion_optimizada.head()


,ID_Operacion,Tipo_Operacion,Departamento,Sede,Responsable,Recurso_Asignado,Prioridad,Estado,Duracion_Estimada_h,Fecha_Programada
0,OP538881,Asignacion_Personal,Logistica,Arequipa,USR1295,Operador,Alta,Pendiente,3,2025-08-05
1,OP432476,Asignacion_Personal,Educacion,Piura,USR9369,Operador,Baja,En Proceso,5,2025-08-13
2,OP500053,Asignacion_Personal,Ingenieria,Cusco,USR6114,Docente,Alta,En Proceso,3,2025-08-04
3,OP624886,Asignacion_Personal,Administracion,Piura,USR6824,Aula,Baja,Completado,8,2025-08-03
4,OP989515,Asignacion_Personal,Sistemas,Cusco,USR5146,Aula,Alta,Completado,7,2025-08-24


## 5. Navegación Autónoma – A* en Grid

In [9]:

df = hojas["Navegacion_Autonoma"]

GRID = 10
inicio = (0, 0)
destino = (9, 9)
obstaculos = {(random.randint(0, 9), random.randint(0, 9)) for _ in range(20)}

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_grid(inicio, destino, obstaculos):
    frontera = [(0 + heuristic(inicio, destino), 0, inicio, [])]
    visitados = set()
    while frontera:
        f, g, actual, camino = heapq.heappop(frontera)
        if actual in visitados:
            continue
        camino = camino + [actual]
        if actual == destino:
            return camino
        visitados.add(actual)
        for dx, dy in [(0,1),(1,0),(-1,0),(0,-1)]:
            nx, ny = actual[0]+dx, actual[1]+dy
            if 0 <= nx < GRID and 0 <= ny < GRID and (nx, ny) not in obstaculos:
                heapq.heappush(frontera, (g+1+heuristic((nx, ny), destino), g+1, (nx, ny), camino))
    return []

camino_robot = a_star_grid(inicio, destino, obstaculos)
camino_robot[:10]


[(0, 0),
 (0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6)]


## 🏁 Conclusión Global

Este caso permitió aplicar algoritmos de planificación heurística a problemas reales:

- 🧠 **Greedy BFS** fue útil para asignaciones sin conflictos.
- 🚛 **A\*** fue eficiente tanto en rutas como en navegación.
- 🧰 **Hill Climbing** optimizó secuencias de producción.
- 👥 **Simulated Annealing** balanceó la carga entre personas.
- 🤖 **A\* en grid** guió un robot en un entorno dinámico.

🧩 Este ejercicio demuestra cómo la inteligencia artificial puede **automatizar operaciones educativas complejas** de manera eficiente.
